### 導入套件 + 設定路徑

In [197]:
import pandas as pd
from pathlib import Path
import glob
import datetime
import re
import os
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from datetime import date
import math


In [198]:
# home and time
home = Path.home()
todaystr = date.today().strftime('%Y-%m-%d')

# set up concat directories
targetFolder = Path(home, 'HP Inc','GPSTW SOP - 2021 日新','Project team','Upload folder ( for buyer update )')
FD_folder = Path(targetFolder, "FD_today","*.xlsx")
shortage_folder = Path(targetFolder ,"shortage_today","*.xlsx")
PNbaseDetail_folder = Path(targetFolder ,"PNbasedDetail_today","*.xlsx")

### FD 合併以及整理

#### 合併檔案

In [199]:
# concat FD
FD_files = []
for FD_file in glob.glob(str(Path(FD_folder))):
    print(FD_file)
    FD = pd.read_excel(FD_file)
    FD_files.append(FD)
FD_all = pd.concat(FD_files)


C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221121_AC ADAPTOR_CQIEC_HsiehAngela_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221121_AC ADAPTOR_CQWIS_HsiehAngela_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221121_AC ADAPTOR_FLH_HsiehAngela_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221121_AC ADAPTOR_TWIEC_HsiehAngela_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221121_BATTERY_CEI_ChangEunice_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221121_BATTERY_CQQCI_ChangEunice_FD.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20221121_BATTERY_CQWIS_ChangEunice_FD.xlsx
C:\Us

In [200]:
# remember inplace = True
FD_all['len_FV'] = FD_all['FV'].str.len()
FD_all['len_platform'] = FD_all['Platform'].str.len()
FD_all = FD_all.sort_values(['len_FV','len_platform'] , ascending=[False,False])
FD_all.reset_index( drop = True, inplace = True )

#### 將max len放到前方

In [201]:
# Index where B is longest
FD_idmax = FD_all['len_platform'].max()
df_max = FD_all.loc[FD_all['len_platform'] == FD_idmax]
df_max_to_add_FD = df_max.head(1)

In [202]:
# isin用在list
# drop the max len row
FD_all = FD_all.drop([FD_all.index[df_max_to_add_FD.index.values[0]]]).reset_index( drop = True )

In [203]:
# concat and put the max len row on the top
FD_concat = [df_max_to_add_FD,FD_all]
FD_output = pd.concat(FD_concat)
FD_output.reset_index( drop = True , inplace = True)

In [204]:
# cut more than 500
FD_output['Platform'] = FD_output['Platform'].apply(lambda x: x[:450] if len(x) > 500 else x)
FD_output = FD_output.drop( columns= ['len_FV','len_platform'])
FD_output

,ODM,Item,Commodity,FV,Platform,HP_PN,Supplier,HP PN,ReportDate,FDdate,FDQty,BuyerName
0,CQQCI,11.0,CONNECTOR,CONN DDR4 260p STD H4,Colts 1.0/Rebak 20C1/Rebak21C1/Langkawi 20C2/L...,"821112-AS4,821112-LS4,821112-FN4,821112-DS5",ARGOSY,821112-AS4,2022-11-21,2022-11-25,161850,LiangLavinia
1,CQIEC,2.0,SSS,(EOL part) SSD 256GB 2280 M2 PCIE 3x2 NVME Val...,Pichu,"L66709-001,L96370-002\nL96370-003",Phison,L96370-003,2022-11-21,2022-11-22,75,LaiShirley
2,CQQCI,5.0,LCD,15.6HD BV LED SVA EDP SLIM 250N NBZ\n15.6HD BV...,Langkawi-1C19/Rebak20C1/Langkawi20C2/Rebak21C1...,"M02097-JD2,M02097-JD1,M02097-LD1,M02097-3D1,M0...",AUO,M02097-3D1,2022-11-21,2022-11-21,5935,ChenJasmine
3,CEI,LCD105,LCD,13.3WUXGA(16:10) AG UWVA EDP1.4PSR 400N DBTS+ ...,Woodford 22C1,"M96425-331,M96425-231,M96425-L31",LGD,M96425-231,2022-11-21,2022-11-24,606,ChenJasmine
4,CQIEC,4.0,LCD,13.3FHD AG LED UWVA EDPPSR USLIM-Bent 400N sRG...,Grizzly 2,"L73329-331,L73329-J31",AUO,L73329-331,2022-11-21,2022-11-24,10,FerngApple
...,...,...,...,...,...,...,...,...,...,...,...,...
457,CQIEC,3.0,AC ADAPTOR,280W WULAI,HOOK2,"M52947-002,M52947-003",CHICONY,M52947-002,2022-11-21,2022-12-15,10368,HsiehAngela
458,CQIEC,3.0,AC ADAPTOR,280W WULAI,HOOK2,"M52947-002,M52947-003",CHICONY,M52947-002,2022-11-21,2022-12-25,10368,HsiehAngela
459,CQIEC,3.0,AC ADAPTOR,280W WULAI,HOOK2,"M52947-002,M52947-003",CHICONY,M52947-002,2022-11-21,2022-12-26,434,HsiehAngela
460,CEI,RAM45,RAM,4GB DDR4,BULLEIT 22C1,L34198-672,MICRON,L34198-672,2022-11-21,2022-11-25,6433,WangWinson(CW)


### Shortage 合併以及整理

#### 合併檔案

In [205]:
# concat shortage
shortage_files = []
for shortage_file in glob.glob(str(Path(shortage_folder))):
    print(shortage_file)
    shortage = pd.read_excel(shortage_file)
    shortage_files.append(shortage)
shortage_all = pd.concat(shortage_files)


C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20221121_AC ADAPTOR_CQIEC_HsiehAngela_Shortage.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20221121_AC ADAPTOR_CQWIS_HsiehAngela_Shortage.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20221121_AC ADAPTOR_FLH_HsiehAngela_Shortage.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20221121_AC ADAPTOR_TWIEC_HsiehAngela_Shortage.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20221121_BATTERY_CEI_ChangEunice_Shortage.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20221121_BATTERY_CQQCI_ChangEunice_Shortage.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for 

In [206]:
shortage_all['len_FV'] = shortage_all['FV'].str.len()
shortage_all['len_platform'] = shortage_all['Platform'].str.len()
shortage_all = shortage_all.sort_values(['len_FV','len_platform'] , ascending=[False,False])
shortage_all.reset_index( drop = True, inplace = True )

#### 將max len放到前面

In [207]:
shortage_idmax = shortage_all['len_platform'].max()
shortage_max = shortage_all.loc[shortage_all['len_platform'] == shortage_idmax]
df_max_to_add_shortage = shortage_max.head(1)
df_max_to_add_shortage

,ODM,Item,Commodity,FV,Platform,HP_PN,P1,Net P2,Net P3,Total Shortage Qty,BT shortage,Working on upside,ReportDate,last FD date,BuyerName,len_FV,len_platform
165,CQQCI,11.0,CONNECTOR,CONN DDR4 260p STD H4,Colts 1.0/Rebak 20C1/Rebak21C1/Langkawi 20C2/L...,"821112-AS4,821112-LS4,821112-FN4,821112-DS5",0,0,170000,170000,0,0,2022-11-21,2022-11-29,LiangLavinia,21,541


In [208]:
# drop the max len row
shortage_all = shortage_all.drop([shortage_all.index[df_max_to_add_shortage.index.values[0]]]).reset_index( drop = True )

In [209]:
# concat and put the max len row on the top
shortage_concat = [ df_max_to_add_shortage , shortage_all ]
shortage_output = pd.concat(shortage_concat)
shortage_output.reset_index( drop = True , inplace = True)

In [210]:
# cut more than 500
shortage_output['Platform'] = shortage_output['Platform'].apply(lambda x: x[:450] if len(x) > 500 else x)
shortage_output = shortage_output.drop( columns= ['len_FV','len_platform'])
shortage_output['Item'] = shortage_output['Item'].astype(str)


### PNbasedDetail 合併及整理

In [211]:
# concat PNbaseDetail
PNbasedDetail_files = []
for PNbasedDetail_file in glob.glob(str(Path(PNbaseDetail_folder))):
    print(PNbasedDetail_file)
    PNbasedDetail = pd.read_excel(PNbasedDetail_file)
    PNbasedDetail_files.append(PNbasedDetail)
PNbaseDetail_all = pd.concat(PNbasedDetail_files)

C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20221121_AC ADAPTOR_CQIEC_HsiehAngela_PNbasedDetail.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20221121_AC ADAPTOR_CQWIS_HsiehAngela_PNbasedDetail.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20221121_AC ADAPTOR_FLH_HsiehAngela_PNbasedDetail.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20221121_AC ADAPTOR_TWIEC_HsiehAngela_PNbasedDetail.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20221121_BATTERY_CEI_ChangEunice_PNbasedDetail.xlsx
C:\Users\panj\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20221121_BATTERY_CQQCI_ChangEunice_PNbasedDetail.xlsx
C:\Users\panj\

In [212]:
try:
    PNbaseDetail_all['len_GPS Remark'] = PNbaseDetail_all['GPS Remark'].str.len()
except:    
    pass
try:
    PNbaseDetail_all['len_ODM use column1'] = PNbaseDetail_all['ODM use column1'].str.len()
except:    
    pass
try:
    PNbaseDetail_all['len_ODM use column2'] = PNbaseDetail_all['ODM use column2'].str.len()
except:    
    pass
try:
    PNbaseDetail_all['len_ODM use column3'] = PNbaseDetail_all['ODM use column3'].str.len()
except:    
    pass
try:
    PNbaseDetail_all['len_ODM use column4'] = PNbaseDetail_all['ODM use column2'].str.len()
except:    
    pass
try:
    PNbaseDetail_all['len_ODM use column5'] = PNbaseDetail_all['ODM use column2'].str.len()
except:    
    pass

In [215]:
PNbaseDetail_all = PNbaseDetail_all.sort_values(['len_GPS Remark','len_ODM use column1','len_ODM use column2','len_ODM use column4','len_ODM use column5'] ,ascending=[True,True,True,True,True])
PNbaseDetail_all.reset_index(drop=True,inplace=True)

In [216]:
PN_idmax = PNbaseDetail_all['len_ODM use column1'].max()
PNbaseDetail_max = PNbaseDetail_all.loc[PNbaseDetail_all['len_ODM use column1'] == PN_idmax]
df_max_to_add_PNbaseDetail = PNbaseDetail_max.head(1)
df_max_to_add_PNbaseDetail

,ODM,Item,Commodity,HP PN,GPS Remark,852 stock,852 stock change,Over pull qty,ODM use column1,ODM use column2,ODM use column3,ODM use column4,ODM use column5,ReportDate,BuyerName,len_GPS Remark,len_ODM use column1,len_ODM use column2,len_ODM use column4,len_ODM use column5
64,CQQCI,11.0,CONNECTOR,821112-AS4,NaN,94609,0,0,/0P1B/0P5F/0P5J/0P5H/0P5N/0P5M/0P9C/0P9F/0PAA/...,Colts 1.0/Rebak 20C1/Rebak21C1/Langkawi 20C2/L...,NaN,NaN,NaN,2022-11-21,LiangLavinia,NaN,215.0,541.0,541.0,541.0


In [217]:
PNbaseDetail_all = PNbaseDetail_all.drop([PNbaseDetail_all.index[df_max_to_add_PNbaseDetail.index.values[0]]]).reset_index( drop = True )

In [218]:
# concat and put the max len row on the top
PNbaseDetail_concat = [ df_max_to_add_PNbaseDetail , shortage_all ]
PNbaseDetail_output = pd.concat(PNbaseDetail_concat)
PNbaseDetail_output.reset_index( drop = True , inplace = True)

### 輸出檔案

In [220]:
# apache airflow to upload SQL ( currently to desktop )
FD_output.to_excel(Path(home, 'Desktop', 'FD_all.xlsx'), index=False)
shortage_output.to_excel(Path(home, 'Desktop', 'Shortage_all.xlsx'), index=False)
PNbaseDetail_all.to_excel(Path(home, 'Desktop', 'PNbaseDetail_all.xlsx'), index=False)